In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os
import urllib.request
import requests

In [ ]:
files = os.listdir("/content/drive/My Drive/leaves/")
path = "/content/drive/My Drive/leaves/" + files[0]
print(os.listdir(path))

In [ ]:
def prep(data_folder_path):
    dirs = os.listdir(data_folder_path)
    
    images = []
    labels = []
  
    for dir_name in dirs:
        print(dir_name)
  
            
        if dir_name=='dry':

          label = 0
          
        elif dir_name=='green':
          label= 1
        
        
        subject_dir_path = data_folder_path + "/" + dir_name
        
        
        subject_images_names = os.listdir(subject_dir_path)
      
        for image_name in subject_images_names:
           
            if image_name.startswith("."):
                continue;
            
           
            image_path = subject_dir_path + "/" + image_name
            

            
            
           
            images.append( image_path)            
            labels.append(label)
            
            
            
    return images,labels

In [ ]:
path = "/content/drive/My Drive/leaves/"
train_img , train_lab = prep(path)

In [ ]:
print(train_lab)

In [ ]:
train_data = []
c=0
for path in train_img:
  img = load_img(path,target_size=(224,224))
  img = img_to_array(img)
  img = preprocess_input(img)
  train_data.append(img)
  c+=1
  print(c)

In [ ]:
trainY = np.array(train_data) 
trainX = np.array(train_lab)

In [ ]:
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [ ]:
baseModel = MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)
model = Model(inputs=baseModel.input, outputs=headModel)
for layer in baseModel.layers:
	layer.trainable = False

In [ ]:
INIT_LR = 1e-4
EPOCHS = 100
BS = 64

opt = Adam(lr=INIT_LR,decay=INIT_LR/EPOCHS)
model.compile(optimizer=opt,loss="sparse_categorical_crossentropy",metrics=["accuracy"])

In [ ]:
from tensorflow import keras
class callbacks(keras.callbacks.Callback):
  def on_epoch_end(self,epochs,logs={}):
    if logs.get('accuracy') > 0.95:
      print("\n Accuracy reached \n")
      self.model.stop_training = True
callbacks = callbacks()

In [ ]:
H = model.fit(
    aug.flow(trainY,trainX),
    steps_per_epoch=len(trainX)//BS,
    epochs=EPOCHS,shuffle=True,callbacks = [callbacks])

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:

def prediction():
  filename = take_photo()
  path = "/content/photo.jpg"
  img = plt.imread(path)
  plt.imshow(img)
  img = load_img(path,target_size=(224,224))
  img = img_to_array(img)
  img = preprocess_input(img)
  img = np.expand_dims(img,axis=0)
  clas = ['dry','green']
  pred = model.predict(img)
  ind = np.argmax(pred[0])
  if ind == 0:
    val = 'd'
  elif ind == 1:
    val = 'g'

  print(clas[ind])
  return clas[ind]

In [ ]:
prediction()